In [ ]:
#coding: utf-8
import numpy as np
import pandas as pd
import re
import jieba
import os
import time
import random
import jieba  #处理中文
import sklearn #分类器
from sklearn.naive_bayes import MultinomialNB  #也可以换成伯努利或高斯的贝叶斯试试看
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt
import time
import gensim
import jieba.analyse
import time

In [ ]:
from gensim.models import CoherenceModel
from gensim import models
def tw_lda_get_tfidf(text,gamma=1.0):
    #1.用正文+标题作为dictionary，过滤极端值
    #--------------下面这里修改
    content_and_title=[i.split(" ") for i in text["微博正文(无标题切词后)"]]
    dictionary = gensim.corpora.Dictionary(content_and_title)
    #-------------下面这里修改
    dictionary.filter_extremes(no_below=15,no_above=0.8, keep_n=100000)
    
    #2.计算标题的tfidf
    #--------------下面这里修改
    processed_docs_title=[i.split(" ") for i in text["标题(切词后)"]]
    bow_corpus_title = [dictionary.doc2bow(doc) for doc in processed_docs_title]
    tfidf_title = models.TfidfModel(bow_corpus_title,normalize=False)
    corpus_tfidf_title = tfidf_title[bow_corpus_title]
#     print("corpus_tfidf_title",corpus_tfidf_title)
    
    #3.计算正文+标题的tfidf
    #--------------下面这里修改
    processed_docs_content=[i.split(" ") for i in text["微博正文(无标题切词后)"]]
    bow_corpus_content = [dictionary.doc2bow(doc) for doc in processed_docs_content]
    tfidf_content = models.TfidfModel(bow_corpus_content,normalize=False)
    corpus_tfidf_content = tfidf_content[bow_corpus_content]
#     print("corpus_tfidf_content",corpus_tfidf_content)
    
    #4.把标题和正文tfidf结合在一起
    new_tfidf=[]
    for i in range(len(corpus_tfidf_content)):#corpus_tfidf2[i]
        dict_2={one:two for one,two in corpus_tfidf_content[i]}
        dict_1={one:two for one,two in corpus_tfidf_title[i]}
        for j in dict_1.keys():
            if j in dict_2.keys():
                dict_2[j]=(1-gamma)*dict_2[j]+gamma*dict_1[j]
        new_tfidf_part=[(one,two) for one,two in sorted(dict_2.items(), key=lambda d: d[0],reverse=False)]
#         print("new_tfidf_part",new_tfidf_part)
#        new_tfidf.append(new_tfidf_part)
#        normed=gensim.models.tfidfmodel.smartirs_normalize(new_tfidf_part,"c")  #-------------这里改了把这个取消归一化了
#        new_tfidf.append(normed)
        new_tfidf.append(new_tfidf_part)
    
    #5.返回tfidf
    return new_tfidf,dictionary


# lda的模型
from gensim.models import CoherenceModel
from gensim import models

def lda_and_coherence_score(processed_docs,num_topics,methods,corpus_tfidf_tw_lda=False,dictionary_tw_lda=False):
    
    if methods=="tw_lda":
        dictionary=dictionary_tw_lda
        corpus=corpus_tfidf_tw_lda
    else:
        dictionary = gensim.corpora.Dictionary(processed_docs)
        dictionary.filter_extremes(no_below=15, no_above=0.8, keep_n=100000)
        bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
        if methods=="lda_normal":
            corpus=bow_corpus
        elif methods=="lda_tfidf":
            tfidf = models.TfidfModel(bow_corpus,normalize=False)#-----------------这里改了，改成false
            corpus_tfidf = tfidf[bow_corpus]
            corpus=corpus_tfidf
    model = gensim.models.LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary,
                                       passes=20,iterations=5000
                                       #,eval_every=1
                                      )
    coherence = CoherenceModel(model=model, texts=processed_docs,dictionary=dictionary, coherence='c_v')
    score = coherence.get_coherence()
    logper=model.log_perplexity(corpus)
    return model,score,logper


#----------增加一步，算5次平均【def名中的10没改，但实际是算5次】
def cs_bar_10(c,num_topics,methods,corpus_tfidf_tw_lda=False,dictionary_tw_lda=False):
    cs_list=[]
    logper_list=[]
    if methods=="tw_lda":
        for i in range(5):
            lda,cs,logper=lda_and_coherence_score(c,num_topics,methods,corpus_tfidf_tw_lda=corpus_tfidf_tw_lda,dictionary_tw_lda=dictionary_tw_lda)
            cs_list.append(cs)
            logper_list.append(logper)
    else:
        for i in range(5):
            lda,cs,logper=lda_and_coherence_score(c,num_topics,methods)
            cs_list.append(cs)
            logper_list.append(logper)
#     cs_bar_10=sum(cs_list)/(len(cs_list)*1.0)               #----------------这里也改了
#     logper_bar_10=sum(logper_list)/(len(logper_list)*1.0)   #----------------这里也改了
    return lda,cs_list,logper_list                            #----------------这里也改了

In [ ]:
text=pd.read_csv("./huanqiu_news_with_title_2.csv")
#看text里有没有空值的项，因为split会发错误
drop_na=[]
for i in range(text.shape[0]):
    if pd.isnull(text.loc[i]).any():
        drop_na.append(i)

text=text.drop(drop_na)
text=text.reset_index(drop=True)

text.head()

# baseline 标准lda和tfidf-lda

有标题切词后

In [ ]:
start_ba=time.process_time()
topic_num=range(10,160,10)
c=[i.split(" ") for i in text["微博正文(有标题切词后)"]]

In [ ]:
cs_nor={}
cs_nor_bar=[]

logper_nor={}
logper_nor_bar=[]

cs_tf={}
cs_tf_bar=[]

logper_tf={}
logper_tf_bar=[]



for i in topic_num:
    starti=time.process_time()
    num_topics=i
    print("i",i)
    
    lda1,cs1,logper1=cs_bar_10(c,num_topics,"lda_normal")
    
    cs_nor[i]=cs1
    cs_nor_bar1=sum(cs1)/(len(cs1)*1.0) 
    cs_nor_bar.append(cs_nor_bar1)
    
    logper_nor[i]=logper1
    logper_bar1=sum(logper1)/(len(logper1)*1.0)
    logper_nor_bar.append(logper_bar1)
    
    
    print("cs_nor",cs1)
    print("cs_nor_bar1",cs_nor_bar1)
    print("logper_nor",logper1)
    print("logper_bar1",logper_bar1)
    
    
    lda2,cs2,logper2=cs_bar_10(c,num_topics,"lda_tfidf")
    
    cs_tf[i]=cs2
    cs_tf_bar2=sum(cs2)/(len(cs2)*1.0) 
    cs_tf_bar.append(cs_tf_bar2)
    
    logper_tf[i]=logper2
    logper_bar2=sum(logper2)/(len(logper2)*1.0)
    logper_tf_bar.append(logper_bar2)
    
    
    print("cs_tf",cs2)
    print("cs_tf_bar2",cs_tf_bar2)
    print("logper_tf",logper2)
    print("logper_bar2",logper_bar2)
    
    endi=time.process_time()
    print('different for topic_num i:',i,'is',endi - starti)
    print("----------------------------------")

    
end_ba = time.process_time()
print('different is %6.3f' % (end_ba - start_ba))

In [ ]:
print("cs_nor")
print(cs_nor)
print("----------------")
print("logper_nor")
print(logper_nor)
print("----------------")
print("cs_tf")
print(cs_tf)
print("----------------")
print("logper_tf")
print(logper_tf)
print("----------------")

In [ ]:
from matplotlib import font_manager
my_font = font_manager.FontProperties(fname='C:/Windows/Fonts/simfang.ttf')

plt.figure(figsize=(8,5),dpi=100)
plt.plot(topic_num,cs_nor_bar,label="Cv_xLDA",marker=".",linewidth=2)
#plt.plot(topic_num,cs_tf_bar,label="Cv_tfidfLDA",marker=".",linewidth=2)
# plt.plot(topic_num,cs_tw_bar,label="cs_tw",marker=".",linewidth=2)

#new_ticks = np.linspace(3, 25, 23)
#plt.xticks(new_ticks)
plt.ylim(0.4, 0.55)

plt.xlabel('TopicNum', fontproperties=my_font)
plt.ylabel('ModelCoherenceScore', fontproperties=my_font)

plt.legend(prop=my_font)

In [ ]:
# 先求per再bar
per_nor_bar=[]
for i in topic_num:
    per_nor_per=[2**((-1)*j) for j in logper_nor[i]]
    temp_per_nor_per=sum(per_nor_per)/(len(per_nor_per)*1.0) 
    per_nor_bar.append(temp_per_nor_per)

In [ ]:
from matplotlib import font_manager
my_font = font_manager.FontProperties(fname='C:/Windows/Fonts/simfang.ttf')

plt.figure(figsize=(8,5),dpi=100)
plt.plot(topic_num,per_nor_bar,label="Perplexity_xLDA",marker=".",linewidth=2)
#plt.plot(topic_num,per_bar_tf,label="Perplexity_tfidfLDA",marker=".",linewidth=2)
# plt.plot(topic_num,cs_tw,label="cs_tw",marker=".",linewidth=2)

#new_ticks = np.linspace(3, 25, 23)
#plt.xticks(new_ticks)
#plt.ylim(0.0, 1.0)

plt.xlabel('TopicNum', fontproperties=my_font)
plt.ylabel('Perplexity', fontproperties=my_font)

plt.legend(prop=my_font)

# 从上面的图中选定topic_num后，进行tw的gamma测试

（cs和perplexity中间选一个作为用来判断topic_num的）

（记得把cs，per的结果列表保存好，放进txt里）

In [ ]:
tn=50 #把确定好的主题数放进这里
c=[i.split(" ") for i in text["微博正文(无标题切词后)"]]

In [ ]:
start_gamma_1=time.process_time()
gamma_l=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
cs_tw_dic={}
cs_tw_bar=[]
logper_tw_dic={}
logper_tw_bar=[]

for gamma in gamma_l:
    start_gamma=time.process_time()
    print("gamma",gamma)
    a,b=tw_lda_get_tfidf(text,gamma=gamma)

    i=tn
    num_topics=i
    print("num_topics is:",i)
        
    lda3,cs3,logper3=cs_bar_10(c,num_topics,"tw_lda",corpus_tfidf_tw_lda=a,dictionary_tw_lda=b)
        
    cs_tw_dic[gamma]=cs3
    cs3_bar=sum(cs3)/(len(cs3)*1.0)
    cs_tw_bar.append(cs3_bar)
    print("cs_tw_list",cs3)
    print("cs3_bar",cs3_bar)

    logper_tw_dic[gamma]=logper3
    logper3_bar=sum(logper3)/(len(logper3)*1.0)
    logper_tw_bar.append(logper3_bar)
    print("logper3",logper3)
    print("logper3_bar",logper3_bar)

    end_gamma=time.process_time()
    print('different for gamma:',gamma,'is',end_gamma-start_gamma)
    print("------------------")
        
end_gamma_1=time.process_time()
print('different for gamma(all):',gamma,'is',end_gamma_1-start_gamma_1)

In [ ]:
#画topicnum=50时，xLDA和tfidfLDA的Cv（这两个是定值，因此是水平的），不同gamma下TWLDA的大小
plt.figure(figsize=(8,5),dpi=100)

#cs_tw_bar[0]=0.4900683903696979

plt.plot(list(cs_tw_dic.keys()),cs_tw_bar,label="Cv_TWLDA",marker=".",linewidth=2)
# for m,n in zip(list(cs_tw_dic.keys()),cs_tw_bar):
#     plt.text(m+0.04, n+0.0015, round(n,5), ha='right', va= 'bottom',fontsize=8)

plt.plot(list(cs_tw_dic.keys()),[cs_nor_bar[4] for i in range(11)],label="Cv_xLDA",linewidth=2)
m=gamma_l[0]
n=cs_tw_bar[4]
# plt.text(m+0.04, n-0.005, round(n,5), ha='right', va= 'bottom',fontsize=8)


plt.plot(list(cs_tw_dic.keys()),[cs_tf_bar[4] for i in range(11)],label="Cv_tfidfLDA",linewidth=2)
m=gamma_l[0]
n=cs_tf_bar[4]
# plt.text(m+0.04, n-0.005, round(n,5), ha='right', va= 'bottom',fontsize=8)



plt.ylim(0.45,0.55)
plt.xlabel('Gamma', fontproperties=my_font)
plt.ylabel('ModelCoherenceScore', fontproperties=my_font)



plt.legend(prop=my_font)

In [ ]:
##不需要----------
##保存模型
# from sklearn.externals import joblib
# #将模型写入 model_joblib 文件
# joblib.dump(lda3, 'twlda_lda3_model3')